In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Importar librerías
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime

#Leer el archivo de los reporter mensaules de MIV Unisalud
df = pd.read_csv('/content/drive/MyDrive/datasets/reportes_bbdd.csv')

In [3]:
# Convertir la columna 'fecha' a datetime
df['fecha'] = pd.to_datetime(df['fecha'], dayfirst=True, errors='coerce')
df['hora'] = pd.to_datetime(df['hora'], format='%H:%M:%S', errors='coerce').dt.time  # Eliminar cualquier dato que no sea hora válida

<ipython-input-3-d8c8b58e67b0>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['fecha'] = pd.to_datetime(df['fecha'], dayfirst=True, errors='coerce')


In [8]:
# Filtrar las horas antes de las 12:00 pm
df_filtered = df[df['hora'] < datetime.strptime('12:00:00', '%H:%M:%S').time()]


# Convertir la hora en minutos de diferencia respecto a las 9:00 am
hora_referencia_area1m = datetime.strptime('09:00:00', '%H:%M:%S').time()
df_filtered['desviacion_minutos'] = df_filtered['hora'].apply(
    lambda t: (datetime.combine(datetime.today(), t) - datetime.combine(datetime.today(), hora_referencia_area1m)).total_seconds() / 60)


# Calcular el RMSE
rmse_area1m = np.sqrt(np.mean(df_filtered['desviacion_minutos']**2))

# Filtrar solo los registros con area = 1
df_area1 = df_filtered[df_filtered['area'] == 1]


# Convertir la hora en minutos desde la medianoche para el área 1
df_area1['hora_minutos'] = df_area1['hora'].apply(lambda t: t.hour * 60 + t.minute + t.second / 60)


# Generar el gráfico de dispersión para área = 1
fig_area1m = px.scatter(df_area1, x='fecha', y='hora_minutos', title=f'Diagrama de dispersión para Área 1 Mañana (RMSE: {rmse_area1m:.2f})',
                 labels={'hora_minutos': 'Hora', 'fecha': 'Fecha'})


# Añadir la línea de referencia en 9:00 am (540 minutos desde medianoche)
fig_area1m.add_shape(type="line", x0=min(df_area1['fecha']), x1=max(df_area1['fecha']), y0=540, y1=540,
              line=dict(color="Red", width=2, dash="dash"))


# Actualizar el espaciado de las fechas para que sea diario
fig_area1m.update_xaxes(dtick="D1")  # Mostrar ticks diarios en el eje X


# Añadir etiquetas en el eje Y en formato HH:MM:SS
fig_area1m.update_layout(
    yaxis=dict(
        tickvals=[480, 540, 600, 660],
        ticktext=["08:00:00", "09:00:00", "10:00:00", "11:00:00"]
    ),
    yaxis_title="Hora del día",
    xaxis_title="Fecha",
    plot_bgcolor='white',  # Fondo del área de trazado
    paper_bgcolor='white'  # Fondo del papel del gráfico
)

# Mostrar gráfico
fig_area1m.show()

# Mostrar RMSE
print(f"=========================================================================")
print(f"RMSE de las desviaciones respecto a las 9:00 am para área 1: {rmse_area1m:.2f} minutos")
print(f"=========================================================================")

<ipython-input-8-724f4f297ce0>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-8-724f4f297ce0>:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



RMSE de las desviaciones respecto a las 9:00 am para área 1: 1.20 minutos


In [9]:
# Filtrar las horas después de las 12:00 pm
df_filtered_tarde = df[df['hora'] > datetime.strptime('12:00:00', '%H:%M:%S').time()]

# Convertir la hora en minutos de diferencia respecto a las 3:00 pm
hora_referencia_area1t = datetime.strptime('15:00:00', '%H:%M:%S').time()
df_filtered_tarde['desviacion_minutos'] = df_filtered_tarde['hora'].apply(
    lambda t: (datetime.combine(datetime.today(), t) - datetime.combine(datetime.today(), hora_referencia_area1t)).total_seconds() / 60)

# Calcular el RMSE
rmse_area1t = np.sqrt(np.mean(df_filtered_tarde['desviacion_minutos']**2))

# Filtrar solo los registros con área = 1
df_area1 = df_filtered_tarde[df_filtered_tarde['area'] == 1]

# Convertir la hora en minutos desde la medianoche para el área 1
df_area1['hora_minutos'] = df_area1['hora'].apply(lambda t: t.hour * 60 + t.minute + t.second / 60)

# Generar el gráfico de dispersión para área = 1
fig_area1t = px.scatter(df_area1, x='fecha', y='hora_minutos', title=f'Diagrama de dispersión para Área 1 Tarde (RMSE: {rmse_area1t:.2f} minutos)',
                 labels={'hora_minutos': 'Hora', 'fecha': 'Fecha'})

# Añadir la línea de referencia en 3:00 pm (900 minutos desde medianoche)
fig_area1t.add_shape(type="line", x0=min(df_area1['fecha']), x1=max(df_area1['fecha']), y0=900, y1=900,
              line=dict(color="Red", width=2))

# Actualizar el espaciado de las fechas para que sea diario
fig_area1t.update_xaxes(dtick="D1")  # Mostrar ticks diarios en el eje X

# Añadir etiquetas en el eje Y en formato HH:MM:SS
fig_area1t.update_layout(
    yaxis=dict(
        tickvals=[840, 900, 960, 1040],
        ticktext=["14:00:00", "15:00:00", "16:00:00", "17:00:00"]
    ),
    yaxis_title="Hora del día",
    xaxis_title="Fecha",
    plot_bgcolor='white',  # Fondo del área de trazado
    paper_bgcolor='white'  # Fondo del papel del gráfico
)

# Establecer el rango del eje Y en minutos
fig_area1t.update_yaxes(range=[897, 903])  # 897 minutos = 15:00, 903 minutos = 15:03

# Mostrar gráfico
fig_area1t.show()

# Mostrar RMSE
print(f"=========================================================================")
print(f"RMSE de las desviaciones respecto a las 3:00 pm para área 1: {rmse_area1t:.2f} minutos")
print(f"=========================================================================")

<ipython-input-9-555b7cfabd1c>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-9-555b7cfabd1c>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



RMSE de las desviaciones respecto a las 3:00 pm para área 1: 1.18 minutos


In [11]:
# Filtrar las horas antes de las 12:00 pm
df_filtered_area2m = df[df['hora'] < datetime.strptime('12:00:00', '%H:%M:%S').time()]


# Convertir la hora en minutos de diferencia respecto a las 9:00 am
hora_referencia_area2m = datetime.strptime('09:00:00', '%H:%M:%S').time()
df_filtered_area2m['desviacion_minutos'] = df_filtered_area2m['hora'].apply(
    lambda t: (datetime.combine(datetime.today(), t) - datetime.combine(datetime.today(), hora_referencia_area2m)).total_seconds() / 60)


# Calcular el RMSE
rmse_area2m = np.sqrt(np.mean(df_filtered_area2m['desviacion_minutos']**2))

# Filtrar solo los registros con area = 2
df_area2 = df_filtered_area2m[df_filtered_area2m['area'] == 2]


# Convertir la hora en minutos desde la medianoche para el área 2
df_area2['hora_minutos'] = df_area2['hora'].apply(lambda t: t.hour * 60 + t.minute + t.second / 60)


# Generar el gráfico de dispersión para área = 2
fig_area2m = px.scatter(df_area2, x='fecha', y='hora_minutos', title=f'Diagrama de dispersión para Área 2 Mañana (RMSE: {rmse:.2f})',
                 labels={'hora_minutos': 'Hora', 'fecha': 'Fecha'})


# Añadir la línea de referencia en 9:00 am (540 minutos desde medianoche)
fig_area2m.add_shape(type="line", x0=min(df_area2['fecha']), x1=max(df_area2['fecha']), y0=540, y1=540,
              line=dict(color="Red", width=2, dash="dash"))


# Actualizar el espaciado de las fechas para que sea diario
fig_area2m.update_xaxes(dtick="D1")  # Mostrar ticks diarios en el eje X


# Añadir etiquetas en el eje Y en formato HH:MM:SS
fig_area2m.update_layout(
    yaxis=dict(
        tickvals=[480, 540, 600, 660],
        ticktext=["08:00:00", "09:00:00", "10:00:00", "11:00:00"]
    ),
    yaxis_title="Hora del día",
    xaxis_title="Fecha",
    plot_bgcolor='white',  # Fondo del área de trazado
    paper_bgcolor='white'  # Fondo del papel del gráfico
)


# Mostrar gráfico
fig_area2m.show()


# Mostrar RMSE
print(f"=========================================================================")
print(f"RMSE de las desviaciones respecto a las 9:00 am para área 2: {rmse_area2m:.2f} minutos")
print(f"=========================================================================")


<ipython-input-11-41910a4df3ec>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-11-41910a4df3ec>:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



RMSE de las desviaciones respecto a las 9:00 am para área 2: 1.20 minutos


In [12]:
# Filtrar las horas después de las 12:00 pm
df_filtered_tarde = df[df['hora'] > datetime.strptime('12:00:00', '%H:%M:%S').time()]

# Convertir la hora en minutos de diferencia respecto a las 3:00 pm
hora_referencia_area2t = datetime.strptime('15:00:00', '%H:%M:%S').time()
df_filtered_tarde['desviacion_minutos'] = df_filtered_tarde['hora'].apply(
    lambda t: (datetime.combine(datetime.today(), t) - datetime.combine(datetime.today(), hora_referencia_area2t)).total_seconds() / 60)

# Calcular el RMSE
rmse_area2t = np.sqrt(np.mean(df_filtered_tarde['desviacion_minutos']**2))

# Filtrar solo los registros con área = 2
df_area2 = df_filtered_tarde[df_filtered_tarde['area'] == 2]

# Convertir la hora en minutos desde la medianoche para el área 2
df_area2['hora_minutos'] = df_area2['hora'].apply(lambda t: t.hour * 60 + t.minute + t.second / 60)

# Generar el gráfico de dispersión para área = 2
fig_area2t = px.scatter(df_area2, x='fecha', y='hora_minutos', title=f'Diagrama de dispersión para Área 2 Tarde (RMSE: {rmse_area1t:.2f} minutos)',
                 labels={'hora_minutos': 'Hora', 'fecha': 'Fecha'})

# Añadir la línea de referencia en 3:00 pm (900 minutos desde medianoche)
fig_area2t.add_shape(type="line", x0=min(df_area2['fecha']), x1=max(df_area2['fecha']), y0=900, y1=900,
              line=dict(color="Red", width=2))

# Actualizar el espaciado de las fechas para que sea diario
fig_area2t.update_xaxes(dtick="D1")  # Mostrar ticks diarios en el eje X

# Añadir etiquetas en el eje Y en formato HH:MM:SS
fig_area2t.update_layout(
    yaxis=dict(
        tickvals=[840, 900, 960, 1040],  # Ejemplo: 840 (14:00), 900 (15:00), 960 (16:00), 1040 (17:00)
        ticktext=["14:00:00", "15:00:00", "16:00:00", "17:00:00"]
    ),
    yaxis_title="Hora del día",
    xaxis_title="Fecha",
    plot_bgcolor='white',  # Fondo del área de trazado
    paper_bgcolor='white'  # Fondo del papel del gráfico
   ## width=1000,  # Ancho del gráfico en píxeles
    #height=800  # Alto del gráfico en píxeles
)

# Establecer el rango del eje Y en minutos
fig_area2t.update_yaxes(range=[897, 903])  # 897 minutos = 15:00, 903 minutos = 15:03

# Mostrar gráfico
fig_area2t.show()

# Mostrar RMSE
print(f"=========================================================================")
print(f"RMSE de las desviaciones respecto a las 3:00 pm para área 2: {rmse_area2t:.2f} minutos")
print(f"=========================================================================")

<ipython-input-12-233b88afebde>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-12-233b88afebde>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



RMSE de las desviaciones respecto a las 3:00 pm para área 2: 1.18 minutos
